In [1]:
import math
import requests, shutil
import os
import re
import csv
import nbformat
import base64
from PIL import Image
from nbformat.v4 import new_markdown_cell

In [2]:
with open(os.path.join("..","p11.ipynb"), encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

len(nb["cells"])

350

In [3]:
plots = {}
qnum = None
for cell in nb["cells"]:
    if cell['cell_type'] == "raw":
        if '# BEGIN QUESTION' in cell['source']:
            qnum = cell['source'].split("\n")[1].split('name:')[1].strip()
        if '# END QUESTION' in cell['source']:
            qnum = None
            
    if cell['cell_type'] == "code" and qnum != None:
        if len(cell['outputs']) == 0:
            continue
        for output in cell['outputs']:
            if 'data' not in output:
                continue
            if 'image/png' not in output['data']:
                continue
            plots[qnum] = output['data']['image/png']
list(plots.keys())

In [4]:
if "images" not in os.listdir(".."):
    os.mkdir(os.path.join("..", "images"))
for plot in plots:
    f = open(os.path.join("..", "images", plot+".jpg"), 'wb')
    f.write(base64.decodebytes(bytes(plots[plot], 'utf-8')))
    f.close()

In [5]:
qnum = None
for idx in range(len(nb['cells'])):
    cell = nb['cells'][idx]
    if cell['cell_type'] == "raw":
        if '# BEGIN QUESTION' in cell['source']:
            qnum = cell['source'].split("\n")[1].split('name:')[1].strip(" -")
        if '# END QUESTION' in cell['source']:
            qnum = None
    
    if qnum == None:
        continue
    if cell['cell_type'] == "markdown" and cell['source'].lower().strip(" \n").endswith('your plot should look like this:'):
        
        markdown_text = '<div style="text-align: center;"><img src="attachment:%s.jpg" style="height: 400px;"/></div>' % (qnum)
        markdown_text += "\n\n<center> <b>Delete</b> this cell before you submit the notebook to reduce the size of your file.</center>"
        img_cell = new_markdown_cell(markdown_text)
        img_cell['attachments'] = {}
        img_cell['attachments']['%s.jpg' % (qnum)] = {}
        img_cell['attachments']['%s.jpg' % (qnum)]['image/jpeg'] = plots[qnum]
        if "attachments" not in nb['cells'][idx+1]:
            nb['cells'].insert(idx+1, img_cell)
        else:
            nb['cells'][idx+1] = img_cell
    
with open(os.path.join("..", "p11.ipynb"), 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)